# 导入需要的包

In [3]:
import xml.etree.ElementTree as ET
import re

# 读入文件

In [27]:
tr1 = ET.ElementTree(file='./data/SS_back.vdx')

In [64]:
ans = {}
for node in tr1.iter():
    patten = '{.*?}'
    label = re.sub(patten,'',node.tag)
    if label == 'StyleSheets':
        for ii in node.getchildren():
            new_label = re.sub(patten,'',ii.tag)
            if new_label == 'StyleSheet':
                tmp = {}
                tmp['ID'] = int(ii.attrib['ID']) #ID
                try:
                    tmp['FillStyle'] = ii.attrib['FillStyle']
                    tmp['FillStyle_exists'] = '1'
                except:
                    tmp['FillStyle_exists'] = '0'
                try:
                    tmp['LineStyle'] = ii.attrib['LineStyle']
                    tmp['LineStyle_exists'] = '1'
                except:
                    tmp['LineStyle_exists'] = '0'
                tmp['Name'] = ii.attrib['Name'] #m名称
                tmp['NameU'] = ii.attrib['NameU']
                if 'TextStyle' in ii.attrib:
                    tmp['TextStyle'] = int(ii.attrib['TextStyle'])
                    tmp['TextStyle_exists'] ='1'
                else:
                    tmp['TextStyle_exists'] ='0'
                ans[ii.attrib['ID']] = tmp

In [65]:
ans

{'0': {'FillStyle_exists': '0',
  'ID': 0,
  'LineStyle_exists': '0',
  'Name': '无样式',
  'NameU': 'No Style',
  'TextStyle_exists': '0'},
 '1': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 1,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': '纯文本',
  'NameU': 'Text Only',
  'TextStyle': 3,
  'TextStyle_exists': '1'},
 '10': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 10,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': 'Gris',
  'NameU': 'Gris',
  'TextStyle': 3,
  'TextStyle_exists': '1'},
 '2': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 2,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': '无',
  'NameU': 'None',
  'TextStyle': 3,
  'TextStyle_exists': '1'},
 '3': {'FillStyle': '0',
  'FillStyle_exists': '1',
  'ID': 3,
  'LineStyle': '0',
  'LineStyle_exists': '1',
  'Name': '正常',
  'NameU': 'Normal',
  'TextStyle': 0,
  'TextStyle_exists': '1'},
 '4': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 4,
  'LineStyle': '3',
  'L

# 定义函数

In [14]:
def Hex_to_RGB(hex):
    hex = hex.replace('#','')
    r = int(hex[0:2],16)
    g = int(hex[2:4],16)
    b = int(hex[4:6], 16)
    rgb = [r,g,b]
    return rgb


分析文件 SS_back.vdx,可以看出来：
   - 主要有以下几个标签：
       1. DocumentProperties
       2. Colors
       3. FaceNames
       4. StyleSheets
       5. DocumentSheet
       6. Masters
       7. Pages
       8. Windows
   - 但是看源c++文件，可以看出只针对：
       1. Colors
       2. FaceNames
       3. Masters
       4. StyleSheets
       5. Pages
   
   进行了解析遍历，暂时先按照源文档进行分析

In [66]:
class vdxFile():
    '''
    vdxFile 主要存储解析出来的vdx各个标签的值
    
    '''
    def __init__(self,filename):
        self.tree = ET.ElementTree(file=filename)
        self.colors = {}
        self.facenames = {}
        self.masters = {}
        self.stylesheets = {}
    #获取颜色信息 Colors标签
    def get_colors(self):
        '''
        通过字典存储colors的信息，key为IX值，RGB通过转换，将16进制转换成10进制。
        例子:
        {'0': [0, 0, 0],
 '1': [255, 255, 255],
 '10': [0, 0, 128],
 '11': [128, 128, 0],...}
        
        '''
        for node in self.tree.iter():
            patten = '{.*?}'
            label = re.sub(patten,'',node.tag)
            if label == 'Colors':
                for ii in node.getchildren():
                    new_label = re.sub(patten,'',ii.tag)
                    if new_label == 'ColorEntry':
                        self.colors[ii.attrib['IX']] = Hex_to_RGB(ii.attrib['RGB'])
    # 获取字体类型FaceNames标签的值 
    def get_facenames(self):
        '''
        获取facenames的相关属性:
        例子:
        {
        '1': {'CharSets': '1614742015 -65536',
  'Flags': '357',
  'ID': 1,
  'Name': 'Arial Unicode MS',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-1 -369098753 63 0'},
 '10': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 10,
  'Name': 'Estrangelo Edessa',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-452972801 -1073717157 41 0'},...
        }
        
        '''
        for node in self.tree.iter():
            patten = '{.*?}'
            label = re.sub(patten,'',node.tag)
            if label == 'FaceNames':
                for ii in node.getchildren():
                    new_label = re.sub(patten,'',ii.tag)
                    if new_label == 'FaceName':
                        tmp = {}
                        tmp['ID'] = int(ii.attrib['ID']) #ID
                        tmp['Name'] = ii.attrib['Name'] #m名称
                        tmp['UnicodeRanges'] = ii.attrib['UnicodeRanges'] #unicode的范围
                        tmp['CharSets'] = ii.attrib['CharSets'] #字符集合
                        tmp['Panos'] = ii.attrib['Panos']
                        tmp['Flags'] = ii.attrib['Flags']
                        self.facenames[ii.attrib['ID']] = tmp
    def get_masters(self):
        '''
        获取master的相关属性
        例子:
        {'0': {'AlignName': '2',
  'BaseID': '{E6E7F788-3308-4917-8165-EE8196E285DA}',
  'Hidden': '0',
  'ID': 0,
  'IconSize': 1,
  'IconUpdate': '1',
  'MatchByName': '0',
  'Name': 'HMI_mtr_vlv_h_d',
  'NameU': 'HMI_mtr_vlv_h_d',
  'PatternFlags': 0,
  'Prompt': '{019D092E-0017-0000-8E40-00608CF305B2}',
  'UniqueID': '0'},...
  }
        '''
        for node in tr1.iter():
            patten = '{.*?}'
            label = re.sub(patten,'',node.tag)
            if label == 'Masters':
                for ii in node.getchildren():
                    new_label = re.sub(patten,'',ii.tag)
                    if new_label == 'Master':
                        tmp = {}
                        tmp['ID'] = int(ii.attrib['ID']) #ID
                        tmp['AlignName'] = ii.attrib['AlignName']
                        tmp['BaseID'] = ii.attrib['BaseID']
                        tmp['Hidden'] = ii.attrib['Hidden']
                        tmp['IconSize'] = int(ii.attrib['IconSize'])
                        tmp['IconUpdate'] = ii.attrib['IconUpdate']
                        tmp['MatchByName'] = ii.attrib['MatchByName']
                        tmp['Name'] = ii.attrib['Name'] #m名称
                        tmp['NameU'] = ii.attrib['NameU']
                        tmp['PatternFlags'] = int(ii.attrib['PatternFlags'])
                        tmp['Prompt'] = ii.attrib['UniqueID']
                        tmp['UniqueID'] = ii.attrib['ID']
                        self.masters[ii.attrib['ID']] = tmp
    def get_stylesheets(self):
        '''
        获取stylesheets的相关属性:
        例子:
        {'0': {'FillStyle_exists': '0',
  'ID': 0,
  'LineStyle_exists': '0',
  'Name': '无样式',
  'NameU': 'No Style',
  'TextStyle_exists': '0'},
 '1': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 1,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': '纯文本',
  'NameU': 'Text Only',
  'TextStyle': 3,
  'TextStyle_exists': '1'},...
  }
        '''
        for node in tr1.iter():
            patten = '{.*?}'
            label = re.sub(patten,'',node.tag)
            if label == 'StyleSheets':
                for ii in node.getchildren():
                    new_label = re.sub(patten,'',ii.tag)
                    if new_label == 'StyleSheet':
                        tmp = {}
                        tmp['ID'] = int(ii.attrib['ID']) #ID
                        try:
                            tmp['FillStyle'] = ii.attrib['FillStyle']
                            tmp['FillStyle_exists'] = '1'
                        except:
                            tmp['FillStyle_exists'] = '0'
                        try:
                            tmp['LineStyle'] = ii.attrib['LineStyle']
                            tmp['LineStyle_exists'] = '1'
                        except:
                            tmp['LineStyle_exists'] = '0'
                        tmp['Name'] = ii.attrib['Name'] #m名称
                        tmp['NameU'] = ii.attrib['NameU']
                        if 'TextStyle' in ii.attrib:
                            tmp['TextStyle'] = int(ii.attrib['TextStyle'])
                            tmp['TextStyle_exists'] ='1'
                        else:
                            tmp['TextStyle_exists'] ='0'
                        self.stylesheets[ii.attrib['ID']] = tmp

In [67]:
mytest = vdxFile('./data/SS_back.vdx')

In [68]:
mytest.get_colors()
mytest.get_facenames()
mytest.get_masters()
mytest.get_stylesheets()

In [69]:
mytest.colors

{'0': [0, 0, 0],
 '1': [255, 255, 255],
 '10': [0, 0, 128],
 '11': [128, 128, 0],
 '12': [128, 0, 128],
 '13': [0, 128, 128],
 '14': [192, 192, 192],
 '15': [230, 230, 230],
 '16': [205, 205, 205],
 '17': [179, 179, 179],
 '18': [154, 154, 154],
 '19': [128, 128, 128],
 '2': [255, 0, 0],
 '20': [102, 102, 102],
 '21': [77, 77, 77],
 '22': [51, 51, 51],
 '23': [26, 26, 26],
 '24': [255, 255, 255],
 '25': [0, 0, 0],
 '26': [222, 222, 222],
 '27': [150, 175, 207],
 '28': [191, 206, 225],
 '29': [192, 80, 70],
 '3': [0, 255, 0],
 '30': [255, 160, 145],
 '31': [255, 185, 174],
 '32': [110, 150, 180],
 '33': [149, 179, 200],
 '34': [73, 121, 192],
 '35': [135, 150, 120],
 '36': [120, 100, 170],
 '37': [30, 30, 30],
 '38': [0, 255, 0],
 '39': [60, 255, 60],
 '4': [0, 0, 255],
 '40': [178, 178, 178],
 '41': [38, 38, 38],
 '42': [200, 200, 200],
 '43': [157, 187, 97],
 '44': [133, 164, 70],
 '45': [0, 176, 80],
 '46': [146, 208, 80],
 '47': [176, 221, 128],
 '48': [255, 0, 0],
 '5': [255, 255, 

In [70]:
mytest.facenames

{'1': {'CharSets': '1614742015 -65536',
  'Flags': '357',
  'ID': 1,
  'Name': 'Arial Unicode MS',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-1 -369098753 63 0'},
 '10': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 10,
  'Name': 'Estrangelo Edessa',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-452972801 -1073717157 41 0'},
 '11': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 11,
  'Name': 'Vrinda',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-452972801 -1073717157 41 0'},
 '12': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 12,
  'Name': 'Shruti',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-452972801 -1073717157 41 0'},
 '13': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 13,
  'Name': 'Mangal',
  'Panos': '2 11 6 4 2 2 2 2 2 4',
  'UnicodeRanges': '-452972801 -1073717157 41 0'},
 '14': {'CharSets': '536936959 539492352',
  'Flags': '327',
  'ID': 14,
  'Name': 'Tunga',
  'Pa

In [71]:
mytest.masters

{'0': {'AlignName': '2',
  'BaseID': '{E6E7F788-3308-4917-8165-EE8196E285DA}',
  'Hidden': '0',
  'ID': 0,
  'IconSize': 1,
  'IconUpdate': '1',
  'MatchByName': '0',
  'Name': 'HMI_mtr_vlv_h_d',
  'NameU': 'HMI_mtr_vlv_h_d',
  'PatternFlags': 0,
  'Prompt': '{019D092E-0017-0000-8E40-00608CF305B2}',
  'UniqueID': '0'},
 '10': {'AlignName': '2',
  'BaseID': '{F9CB96C1-AC6E-4F11-8B72-CAC7C736E712}',
  'Hidden': '0',
  'ID': 10,
  'IconSize': 1,
  'IconUpdate': '1',
  'MatchByName': '0',
  'Name': 'HMI_navigate1',
  'NameU': 'HMI_navigate1',
  'PatternFlags': 0,
  'Prompt': '{01220B99-0007-0000-8E40-00608CF305B2}',
  'UniqueID': '10'},
 '11': {'AlignName': '2',
  'BaseID': '{54A3AC5D-2916-446D-B2BB-B2E756DEB497}',
  'Hidden': '0',
  'ID': 11,
  'IconSize': 1,
  'IconUpdate': '1',
  'MatchByName': '0',
  'Name': 'mtr_but_vlv_h_c.38',
  'NameU': 'mtr_but_vlv_h_c.38',
  'PatternFlags': 0,
  'Prompt': '{014C67B4-0034-0000-8E40-00608CF305B2}',
  'UniqueID': '11'},
 '12': {'AlignName': '2',
  '

In [72]:
mytest.stylesheets

{'0': {'FillStyle_exists': '0',
  'ID': 0,
  'LineStyle_exists': '0',
  'Name': '无样式',
  'NameU': 'No Style',
  'TextStyle_exists': '0'},
 '1': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 1,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': '纯文本',
  'NameU': 'Text Only',
  'TextStyle': 3,
  'TextStyle_exists': '1'},
 '10': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 10,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': 'Gris',
  'NameU': 'Gris',
  'TextStyle': 3,
  'TextStyle_exists': '1'},
 '2': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 2,
  'LineStyle': '3',
  'LineStyle_exists': '1',
  'Name': '无',
  'NameU': 'None',
  'TextStyle': 3,
  'TextStyle_exists': '1'},
 '3': {'FillStyle': '0',
  'FillStyle_exists': '1',
  'ID': 3,
  'LineStyle': '0',
  'LineStyle_exists': '1',
  'Name': '正常',
  'NameU': 'Normal',
  'TextStyle': 0,
  'TextStyle_exists': '1'},
 '4': {'FillStyle': '3',
  'FillStyle_exists': '1',
  'ID': 4,
  'LineStyle': '3',
  'L